In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import mynnlib
from mynnlib import *

dataset_dir = "insect-dataset/root-classifier"

# Create datasets

In [25]:
target_data_cnt_per_class = 15000

for class_name in ["moth", "butterfly"]:
    class_dir = f"{dataset_dir}/data/{class_name}"
    if os.path.exists(class_dir):
        shutil.rmtree(class_dir)
    src_dir = f"{dataset_dir}/../{class_name}/data"
    data_cnt = sum([len(os.listdir(f"{src_dir}/{species_name}")) for species_name in os.listdir(src_dir)])
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)
    used_data_cnt = 0
    for species_name in os.listdir(src_dir):
        for file in Path(f"{src_dir}/{species_name}").iterdir():
            if file.is_file() and random.random() <= target_data_cnt_per_class/data_cnt:
                used_data_cnt += 1
                shutil.copy(file, f"{class_dir}/{file.name}")
    print(f"class: {class_name:20} | total data count: {data_cnt:10} | used data count: {used_data_cnt:10}")

class: moth                 | total data count:     187506 | used data count:      14779
class: butterfly            | total data count:     124809 | used data count:      14967


In [26]:
other_data_cnt = sum([1 for class_name in os.listdir(f"{dataset_dir}/data/other")])
print(f"class: {'other':20} | used data count: {other_data_cnt:10}")

class: other                | used data count:      14070


In [27]:
other_insect_data_cnt = sum([1 for class_name in os.listdir(f"{dataset_dir}/data/other-insect")])
print(f"class: {'other':20} | used data count: {other_insect_data_cnt:10}")

class: other                | used data count:       3896


# Train

### Model A (resnet-18)

In [29]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet18", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.root-classifier.ta.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 0.1766 Acc: 0.9332  | Val Loss: 0.2624 Acc: 0.9305  | Elapsed time: 0:05:49.368873
Epoch    2 /    5  | Train Loss: 0.0988 Acc: 0.9624  | Val Loss: 0.3145 Acc: 0.9109  | Elapsed time: 0:11:26.323126
Phase 2:
Epoch    1 /    5  | Train Loss: 0.2937 Acc: 0.8861  | Val Loss: 0.1139 Acc: 0.9626  | Elapsed time: 0:06:53.654441
Epoch    2 /    5  | Train Loss: 0.2317 Acc: 0.9092  | Val Loss: 0.1865 Acc: 0.9465  | Elapsed time: 0:14:57.317811
Phase 3:
Epoch    1 /    5  | Train Loss: 0.2392 Acc: 0.9052  | Val Loss: 0.3843 Acc: 0.9127  | Elapsed time: 0:05:30.800892
Epoch    2 /    5  | Train Loss: 0.2203 Acc: 0.9146  | Val Loss: 0.1586 Acc: 0.9537  | Elapsed time: 0:11:04.983225
Epoch    3 /    5  | Train Loss: 0.2121 Acc: 0.9169  | Val Loss: 0.1383 Acc: 0.9554  | Elapsed time: 0:16:37.199918
Phase 4:
Epoch    1 /    5  | Train Loss: 0.1971 Acc: 0.9222  | Val Loss: 0.1510 Acc: 0.9519  | Elapsed time: 0:05:31.102094
Epoch    2 /    5  | Train Loss: 0.1

In [30]:
model_data = torch.load(f"{dataset_dir}/checkpoint.root-classifier.ta.ep050001.pth", weights_only=False)

In [41]:
test_class(model_data, f"{dataset_dir}/val", ["moth", "butterfly", "other"])

Class: moth            ----> Success: 143/157 -> 0.91%
Class: butterfly       ----> Success: 314/331 -> 0.95%
Class: other           ----> Success: 542/561 -> 0.97%


### Model B (resnet-18)
added other-insect class

In [46]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet18", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.root-classifier.tb.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 0.2307 Acc: 0.9159  | Val Loss: 0.1703 Acc: 0.9458  | Elapsed time: 0:09:20.039842
Epoch    2 /    5  | Train Loss: 0.1274 Acc: 0.9541  | Val Loss: 0.1647 Acc: 0.9445  | Elapsed time: 0:14:37.725237
Phase 2:
Epoch    1 /    5  | Train Loss: 0.3708 Acc: 0.8613  | Val Loss: 0.1759 Acc: 0.9433  | Elapsed time: 0:08:59.943711
Epoch    2 /    5  | Train Loss: 0.2904 Acc: 0.8883  | Val Loss: 0.1090 Acc: 0.9647  | Elapsed time: 0:15:22.880138
Epoch    3 /    5  | Train Loss: 0.2669 Acc: 0.8988  | Val Loss: 0.1150 Acc: 0.9697  | Elapsed time: 0:21:23.838444
Phase 3:
Epoch    1 /    5  | Train Loss: 0.2877 Acc: 0.8902  | Val Loss: 0.1485 Acc: 0.9521  | Elapsed time: 0:08:24.598403
Epoch    2 /    5  | Train Loss: 0.2695 Acc: 0.8981  | Val Loss: 0.1888 Acc: 0.9407  | Elapsed time: 0:15:23.914549
Phase 4:
Epoch    1 /    5  | Train Loss: 0.2578 Acc: 0.9031  | Val Loss: 0.1462 Acc: 0.9571  | Elapsed time: 0:06:00.299434
Epoch    2 /    5  | Train Loss: 0.2

In [47]:
model_data = torch.load(f"{dataset_dir}/checkpoint.root-classifier.tb.ep050002.pth", weights_only=False)

In [8]:
test_class(model_data, f"{dataset_dir}/val", ["moth", "butterfly", "other-insect", "other"])

Class: moth            ----> Success: 138/157 -> 87.90%
Class: butterfly       ----> Success: 170/174 -> 97.70%
Class: other-insect    ----> Success: 229/232 -> 98.71%
Class: other           ----> Success: 228/230 -> 99.13%


In [9]:
test_class(model_data, f"{dataset_dir}/test", ["moth", "butterfly", "other-insect", "other"])

Class: moth            ----> Success: 8/11 -> 72.73%
Class: butterfly       ----> Success: 23/24 -> 95.83%
Class: other-insect    ----> Success: 9/15 -> 60.00%
Class: other           ----> Success: 6/17 -> 35.29%


In [48]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/test", ["moth", "butterfly", "other-insect", "other"], 0.75)

Class: moth            ----> Success: 7/11 -> 63.64%
Class: butterfly       ----> Success: 22/24 -> 91.67%
Class: other-insect    ----> Success: 5/15 -> 33.33%
Class: other           ----> Success: 3/17 -> 17.65%


In [50]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/test", ["moth", "butterfly", "other-insect", "other"], 0.3)

Class: moth            ----> Success: 10/11 -> 90.91%
Class: butterfly       ----> Success: 23/24 -> 95.83%
Class: other-insect    ----> Success: 9/15 -> 60.00%
Class: other           ----> Success: 7/17 -> 41.18%


In [51]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/val", ["moth", "butterfly", "other-insect", "other"], 0.3)

Class: moth            ----> Success: 142/157 -> 90.45%
Class: butterfly       ----> Success: 170/174 -> 97.70%
Class: other-insect    ----> Success: 230/232 -> 99.14%
Class: other           ----> Success: 230/230 -> 100.00%


### Model C (resnet-34)

In [6]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet34", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.root-classifier.tc.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 0.2218 Acc: 0.9199  | Val Loss: 0.1400 Acc: 0.9533  | Elapsed time: 0:10:11.404796
Epoch    2 /    5  | Train Loss: 0.1332 Acc: 0.9518  | Val Loss: 0.1557 Acc: 0.9458  | Elapsed time: 0:15:51.321329
Phase 2:
Epoch    1 /    5  | Train Loss: 0.3595 Acc: 0.8639  | Val Loss: 0.1643 Acc: 0.9458  | Elapsed time: 0:06:10.336970
Epoch    2 /    5  | Train Loss: 0.2924 Acc: 0.8900  | Val Loss: 0.1267 Acc: 0.9622  | Elapsed time: 0:12:32.109651
Phase 3:
Epoch    1 /    5  | Train Loss: 0.3012 Acc: 0.8854  | Val Loss: 0.2874 Acc: 0.9105  | Elapsed time: 0:06:24.290519
Epoch    2 /    5  | Train Loss: 0.2779 Acc: 0.8949  | Val Loss: 0.1541 Acc: 0.9458  | Elapsed time: 0:12:55.787255
Epoch    3 /    5  | Train Loss: 0.2654 Acc: 0.9001  | Val Loss: 0.1812 Acc: 0.9382  | Elapsed time: 0:19:23.164769
Phase 4:
Epoch    1 /    5  | Train Loss: 0.2382 Acc: 0.9097  | Val Loss: 0.1429 Acc: 0.9508  | Elapsed time: 0:06:44.090558
Epoch    2 /    5  | Train Loss: 0.2

In [28]:
model_data = torch.load(f"{dataset_dir}/checkpoint.root-classifier.tc.ep050003.pth", weights_only=False)

In [15]:
test_class(model_data, f"{dataset_dir}/val", ["moth", "butterfly", "other-insect", "other"])

Class: moth            ----> Success: 141/157 -> 89.81%
Class: butterfly       ----> Success: 172/174 -> 98.85%
Class: other-insect    ----> Success: 231/232 -> 99.57%
Class: other           ----> Success: 228/230 -> 99.13%


In [16]:
test_class(model_data, f"{dataset_dir}/test", ["moth", "butterfly", "other-insect", "other"])

Class: moth            ----> Success: 8/11 -> 72.73%
Class: butterfly       ----> Success: 24/24 -> 100.00%
Class: other-insect    ----> Success: 4/15 -> 26.67%
Class: other           ----> Success: 10/17 -> 58.82%


In [40]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/test", ["moth", "butterfly", "other-insect", "other"], 0.75)

Class: moth            ----> Success: 7/11 -> 63.64%
Class: butterfly       ----> Success: 22/24 -> 91.67%
Class: other-insect    ----> Success: 3/15 -> 20.00%
Class: other           ----> Success: 5/17 -> 29.41%


In [46]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/test", ["moth", "butterfly", "other-insect", "other"], 0.3)

Class: moth            ----> Success: 8/11 -> 72.73%
Class: butterfly       ----> Success: 24/24 -> 100.00%
Class: other-insect    ----> Success: 7/15 -> 46.67%
Class: other           ----> Success: 10/17 -> 58.82%


In [44]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/val", ["moth", "butterfly", "other-insect", "other"], 0.3)

Class: moth            ----> Success: 144/157 -> 91.72%
Class: butterfly       ----> Success: 174/174 -> 100.00%
Class: other-insect    ----> Success: 232/232 -> 100.00%
Class: other           ----> Success: 228/230 -> 99.13%


In [29]:
# another random subset of original butterfly/moth data
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/data", ["moth", "butterfly", "other-insect", "other"], 0.3)

Class: moth            ----> Success: 13904/14777 -> 94.09%
Class: butterfly       ----> Success: 14675/14937 -> 98.25%
Class: other-insect    ----> Success: 3784/3896 -> 97.13%
Class: other           ----> Success: 13948/14070 -> 99.13%


### Model D (resnet-50)

In [97]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet50", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.root-classifier.td.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 0.1757 Acc: 0.9402  | Val Loss: 0.1463 Acc: 0.9559  | Elapsed time: 0:10:32.551645
Epoch    2 /    5  | Train Loss: 0.0778 Acc: 0.9727  | Val Loss: 0.1186 Acc: 0.9660  | Elapsed time: 0:17:50.348645
Phase 2:
Epoch    1 /    5  | Train Loss: 0.2593 Acc: 0.9019  | Val Loss: 0.1139 Acc: 0.9672  | Elapsed time: 0:07:55.200823
Epoch    2 /    5  | Train Loss: 0.2036 Acc: 0.9240  | Val Loss: 0.1479 Acc: 0.9533  | Elapsed time: 0:15:39.972649
Phase 3:
Epoch    1 /    5  | Train Loss: 0.2017 Acc: 0.9253  | Val Loss: 0.1114 Acc: 0.9622  | Elapsed time: 0:07:38.059949
Epoch    2 /    5  | Train Loss: 0.1870 Acc: 0.9304  | Val Loss: 0.0923 Acc: 0.9672  | Elapsed time: 0:15:16.613885
Phase 4:
Epoch    1 /    5  | Train Loss: 0.2001 Acc: 0.9254  | Val Loss: 0.2211 Acc: 0.9357  | Elapsed time: 0:07:41.252944
Epoch    2 /    5  | Train Loss: 0.1550 Acc: 0.9423  | Val Loss: 0.1107 Acc: 0.9647  | Elapsed time: 0:15:45.025357
Epoch    3 /    5  | Train Loss: 0.1

In [28]:
model_data = torch.load(f"{dataset_dir}/checkpoint.root-classifier.td.ep050000.pth", weights_only=False)

In [22]:
test_class(model_data, f"{dataset_dir}/val", ["moth", "butterfly", "other-insect", "other"])

Class: moth            ----> Success: 138/157 -> 87.90%
Class: butterfly       ----> Success: 172/174 -> 98.85%
Class: other-insect    ----> Success: 227/232 -> 97.84%
Class: other           ----> Success: 230/230 -> 100.00%


In [23]:
test_class(model_data, f"{dataset_dir}/test", ["moth", "butterfly", "other-insect", "other"])

Class: moth            ----> Success: 9/11 -> 81.82%
Class: butterfly       ----> Success: 24/24 -> 100.00%
Class: other-insect    ----> Success: 9/15 -> 60.00%
Class: other           ----> Success: 9/17 -> 52.94%


In [29]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/test", ["moth", "butterfly", "other-insect", "other"], 0.75)

Class: moth            ----> Success: 7/11 -> 63.64%
Class: butterfly       ----> Success: 23/24 -> 95.83%
Class: other-insect    ----> Success: 6/15 -> 40.00%
Class: other           ----> Success: 6/17 -> 35.29%


In [54]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/test", ["moth", "butterfly", "other-insect", "other"], 0.3)

Class: moth            ----> Success: 10/11 -> 90.91%
Class: butterfly       ----> Success: 23/24 -> 95.83%
Class: other-insect    ----> Success: 9/15 -> 60.00%
Class: other           ----> Success: 7/17 -> 41.18%


In [38]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/val", ["moth", "butterfly", "other-insect", "other"], 0.3)

Class: moth            ----> Success: 142/157 -> 90.45%
Class: butterfly       ----> Success: 174/174 -> 100.00%
Class: other-insect    ----> Success: 227/232 -> 97.84%
Class: other           ----> Success: 230/230 -> 100.00%


### Model E (resnet-34 + new subset dataset)

Model C struggling with brown side photos of butterfly with moth

new train dataset = new random subset of butterfly/moth dataset + butterfly/moth test dataset

In [7]:
training_params = [
    { "idx": 1, "robustness": 0.2, "break_at_val_acc_diff": 0.05},
    { "idx": 2, "robustness": 0.5, "break_at_val_acc_diff": 0.02},
    { "idx": 3, "robustness": 1.0, "break_at_val_acc_diff": 0.01},
    { "idx": 4, "robustness": 2.0, "break_at_val_acc_diff": -0.000001},
    { "idx": 5, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    if param["idx"] == 1:
        model_data = init_model_for_training(f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                             batch_size=32, arch="resnet34", image_size=224, robustness=param["robustness"],
                                             lr=1e-4, weight_decay=1e-4, silent=True)
    else:
        model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                            batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.root-classifier.te.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 1:
Epoch    1 /    5  | Train Loss: 0.2247 Acc: 0.9180  | Val Loss: 0.2399 Acc: 0.9069  | Elapsed time: 0:08:52.739502
Epoch    2 /    5  | Train Loss: 0.1389 Acc: 0.9503  | Val Loss: 0.1839 Acc: 0.9279  | Elapsed time: 0:14:26.033736
Phase 2:
Epoch    1 /    5  | Train Loss: 0.3543 Acc: 0.8665  | Val Loss: 0.3720 Acc: 0.8689  | Elapsed time: 0:06:01.955695
Epoch    2 /    5  | Train Loss: 0.2882 Acc: 0.8904  | Val Loss: 0.2382 Acc: 0.9187  | Elapsed time: 0:12:06.001599
Epoch    3 /    5  | Train Loss: 0.2622 Acc: 0.9013  | Val Loss: 0.3082 Acc: 0.8873  | Elapsed time: 0:18:55.013008
Phase 3:
Epoch    1 /    5  | Train Loss: 0.2836 Acc: 0.8933  | Val Loss: 0.2519 Acc: 0.9161  | Elapsed time: 0:09:16.898676
Epoch    2 /    5  | Train Loss: 0.2619 Acc: 0.9009  | Val Loss: 0.4518 Acc: 0.8558  | Elapsed time: 0:15:31.199762
Phase 4:
Epoch    1 /    5  | Train Loss: 0.2363 Acc: 0.9110  | Val Loss: 0.2141 Acc: 0.9266  | Elapsed time: 0:06:19.370162
Epoch    2 /    5  | Train Loss: 0.2

In [11]:
training_params = [
    { "idx": 6, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                        batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.root-classifier.te.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 6:
Epoch    1 /    5  | Train Loss: 0.1709 Acc: 0.9345  | Val Loss: 0.1534 Acc: 0.9463  | Elapsed time: 0:10:53.020002
Epoch    2 /    5  | Train Loss: 0.1630 Acc: 0.9387  | Val Loss: 0.1434 Acc: 0.9476  | Elapsed time: 0:17:24.863814
Epoch    3 /    5  | Train Loss: 0.1636 Acc: 0.9383  | Val Loss: 0.1472 Acc: 0.9502  | Elapsed time: 0:23:53.039336
Epoch    4 /    5  | Train Loss: 0.1624 Acc: 0.9395  | Val Loss: 0.1403 Acc: 0.9528  | Elapsed time: 0:30:20.622806
Epoch    5 /    5  | Train Loss: 0.1615 Acc: 0.9381  | Val Loss: 0.1441 Acc: 0.9515  | Elapsed time: 0:36:47.629922


In [12]:
training_params = [
    { "idx": 7, "robustness": 2.0, "break_at_val_acc_diff": -0.000001}
]
for param in training_params:
    print(f"Phase {param["idx"]}:")
    model_data = prepare_for_retraining(model_data, f'{dataset_dir}/data', f'{dataset_dir}/val', 
                                        batch_size=32, image_size=224, robustness=param["robustness"], silent=True)
    train(model_data, 5, f"{dataset_dir}/checkpoint.root-classifier.te.ep{param["idx"]:02}###.pth", 
          break_at_val_acc_diff=param["break_at_val_acc_diff"])

Phase 7:
Epoch    1 /    5  | Train Loss: 0.1602 Acc: 0.9397  | Val Loss: 0.1459 Acc: 0.9489  | Elapsed time: 0:05:56.401994
Epoch    2 /    5  | Train Loss: 0.1594 Acc: 0.9393  | Val Loss: 0.1570 Acc: 0.9450  | Elapsed time: 0:12:18.342143


In [30]:
model_data = torch.load(f"{dataset_dir}/checkpoint.root-classifier.te.ep060004.pth", weights_only=False)

In [14]:
test_class(model_data, f"{dataset_dir}/val", ["moth", "butterfly", "other-insect", "other"])

Class: moth            ----> Success: 139/157 -> 88.54%
Class: butterfly       ----> Success: 171/174 -> 98.28%
Class: other-insect    ----> Success: 189/202 -> 93.56%
Class: other           ----> Success: 227/230 -> 98.70%


In [20]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/test", ["moth", "butterfly", "other-insect", "other"], 0.3)

Class: moth            ----> Success: 11/11 -> 100.00%
Class: butterfly       ----> Success: 24/24 -> 100.00%
Class: other-insect    ----> Success: 12/15 -> 80.00%
Class: other           ----> Success: 12/17 -> 70.59%


In [15]:
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/val", ["moth", "butterfly", "other-insect", "other"], 0.3)

Class: moth            ----> Success: 142/157 -> 90.45%
Class: butterfly       ----> Success: 172/174 -> 98.85%
Class: other-insect    ----> Success: 194/202 -> 96.04%
Class: other           ----> Success: 227/230 -> 98.70%


In [31]:
# another random subset of original butterfly/moth data
test_class_for_min_accepted_softmax(model_data, f"{dataset_dir}/data", ["moth", "butterfly", "other-insect", "other"], 0.3)

Class: moth            ----> Success: 14040/14777 -> 95.01%
Class: butterfly       ----> Success: 14629/14937 -> 97.94%
Class: other-insect    ----> Success: 3788/3896 -> 97.23%
Class: other           ----> Success: 13956/14070 -> 99.19%
